In [1]:
import pandas as pd
import numpy as np
import re
from completing_missing_clinician_questionnaire_2021 import map_additional_column_name_to_2021_clinician_column_name, should_remove_from_stu_columns, map_additional_column_name_to_2021_student_column_name , fill_id, IsClinician, upload_to_imputation_data_to_old_data_map, upload_to_old_data_column_names_map
import psycopg2

conn_str = "dbname=postgres user=postgres password=asdf123"

In [2]:
student_questionnaires = ["opening_students", "trq_sf_maris_stu", "remote_stu", "cps_stu", "scs_stu", "mini_kid_sum_stu", "c_ssrs_stu"]

In [3]:

clinician_start = 'opening_clinicians_complete'
clinician_end_2021_2022 = 'cssrs_t_clin_complete'#, 'cssrs_fw_maya_complete'
clinician_end_add = 'cssrs_t_complete'

student_start = 'opening_students_timestamp'
student_end = 'cssrs_t_stu_complete'


In [4]:
conn = psycopg2.connect(conn_str)
sql = "SELECT * FROM auxiliary_questionnaires_data.questionnaires_columns_names;"
df = pd.read_sql_query(sql, conn)
conn.close()

In [8]:
old_data = pd.read_csv(r"../../Data/OriginalDataset/Schneider Depression Clinic Database.csv", na_values= ' ')
redcap_data = pd.read_csv(r"../../Data/OriginalDataset/ImmiRiskIPT2022_DATA_2023-09-03_1503.csv", na_values= ' ')
imputation_data = pd.read_csv(r"../../Data/helper_docs/Student_Clinician_data_2021.csv", na_values= ' ')
# the warning rooted from the nan values


C:\Users\nogur\anaconda3\envs\DeppClinic\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (44,46,51,53,99,325,366,439,453,459,464,466,501,507,512,514,519,521,530,532,569,571,580,599,766,771,773,779,784,789,794,798,800,804,809,811,813,816,818,827,829,849,866,868,877,896,906,941,964,1002,1014,1025,1063,1068,1070,1076,1081,1086,1091,1093,1095,1097,1099,1100,1126,1142,1149,1165,1168,1172,1173,1177,1178,1179,1180,1182,1183,1184,1187,1188,1190,1207,1212,1229,1232,1260,1262,1264,1266,1268,1271,1281,1283,1285,1291,1293,1297,1305,1306,1319,1321,1324,1326,1338,1340,1342,1343,1344,1350,1351,1352,1353,1465,1467,1468,1470,1486,1489,1492,1496,1513,1515,1516,1517,1523,1524,1525,1526,1580,1650,1653,1654) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:


rename_imputation_data = {
    'cssrs_followup_timestamp':'cssrs_fw_maya_timestamp',
    'cssrs_followup_complete':'cssrs_fw_maya_complete',
}

rename_old_data = {
    'cssrs_followup_timestamp':'cssrs_fw_maya_timestamp',
    'cssrs_followup_complete':'cssrs_fw_maya_complete',
    'chameleon_complete_stu': 'chameleon_complete'
}

## error record_id = 363
imputation_data = imputation_data[imputation_data['record_id']!=363]

#do the imputation id like of redcap dataset
imputation_data = fill_id(imputation_data)


imputation_data = imputation_data.rename(rename_imputation_data, axis=1)

old_data = old_data.rename(rename_old_data, axis=1)



In [10]:
old_data_column_names = list(old_data.columns)

redcap_data_column_names = list(redcap_data.columns)
imputation_data_column_names = list(imputation_data.columns)


# Student column: in old dataset and not in redcap 
----

In [11]:
### student columns

old_data_student_columns = old_data_column_names[old_data_column_names.index(student_start):old_data_column_names.index(student_end)+1]
redcap_data_student_columns = redcap_data_column_names[redcap_data_column_names.index(student_start):redcap_data_column_names.index(student_end)+1] 

imputation_data_student_columns = imputation_data_column_names[imputation_data_column_names.index(clinician_start):imputation_data_column_names.index(clinician_end_add)+1]
imputation_data_student_columns = imputation_data_student_columns + ['who', 'who_other', 'name']

imputation_data_student_columns = [i for i in imputation_data_student_columns if not should_remove_from_stu_columns(i)]



# imputation -> old data

In [12]:
imputation_2_old_map = {
#     # imputation_data_clinician_columns : redcap_data_clinician_columns
#     'trqsfmaris_timestamp': 'trqsfmarisclin_timestamp',
#     'trqsfmaris_complete': 'trqsfmarisclin_complete'
}


for student_column in imputation_data_student_columns:
    new_name, success = map_additional_column_name_to_2021_student_column_name(student_column, old_data_student_columns)
    if success:
        imputation_2_old_map[student_column] = new_name
    else:
        print(student_column)

# Old data -> redcap data

In [13]:
old_2_redcap_map = {
        # redcap_data_clinician_columns: 
}

for student_column in old_data_student_columns:
    new_name, success = map_additional_column_name_to_2021_student_column_name(student_column, redcap_data_student_columns)
    if success:
        old_2_redcap_map[student_column] = new_name
    else:
        print(student_column)


In [14]:
len(imputation_2_old_map.keys())

88

In [15]:
len(old_2_redcap_map.keys())

156

In [16]:
len(old_data_student_columns)

156

In [17]:
len(imputation_data_student_columns)

88

In [18]:
len(redcap_data_student_columns)

173

In [23]:
for questionnaire in student_questionnaires:
    
    redcap_columns = df.loc[df['questionnaire_name'] == questionnaire, 'column_names'].values[0]
    questionnaire_old_2_redcap_map = {key: old_2_redcap_map[key] for key in redcap_columns if key in old_2_redcap_map}
    old_data_columns = list(questionnaire_old_2_redcap_map.values())
    questionnaire_imputation_2_old_map = {key: imputation_2_old_map[key] for key in old_data_columns if key in imputation_2_old_map}
    print(questionnaire, '\n')
    print(old_data_columns, '\n\n')
    print('\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\')
    
#     upload_to_old_data_column_names_map(conn_str, questionnaire, questionnaire_old_2_redcap_map)
    
#     upload_to_imputation_data_to_old_data_map(conn_str, questionnaire, questionnaire_imputation_2_old_map)

opening_students 

['opening_students_timestamp', 'who_stu', 'who_other_stu', 'name_stu'] 


\\\\\\\\\\\\\\\\\\\\\\
trq_sf_maris_stu 

['trqsfmaris_stu_timestamp', 'trq_sf_maris_1_stu', 'trq_sf_maris_2_stu', 'trq_sf_maris_3_stu', 'trq_sf_maris_4_stu', 'trq_sf_maris_5_stu', 'trq_sf_maris_6_stu', 'trq_sf_maris_7_stu', 'trq_sf_maris_8_stu', 'trq_sf_maris_9_stu', 'trq_sf_maris_10_stu', 'trq_sf_maris_11_stu', 'trq_sf_maris_12_stu', 'trq_sf_maris_13_stu', 'trq_sf_maris_14_stu'] 


\\\\\\\\\\\\\\\\\\\\\\
remote_stu 

['remote_stu_timestamp', 'remote_1_stu'] 


\\\\\\\\\\\\\\\\\\\\\\
cps_stu 

['cps_stu_timestamp', 'cps_1_stu', 'cps_2_stu'] 


\\\\\\\\\\\\\\\\\\\\\\
scs_stu 

['sci_c_1_1_stu', 'scs_2_1_stu', 'scs_2_2_stu', 'scs_2_3_stu', 'scs_2_4_stu', 'sci_c_3_1_stu', 'sci_c_3_2_stu', 'sci_c_3_3_stu', 'sci_c_3_4_stu', 'sci_c_4_1_stu', 'sci_c_4_2_stu', 'sci_c_4_3_stu', 'sci_c_4_4_stu', 'sci_c_5_1_stu', 'sci_c_5_2_stu'] 


\\\\\\\\\\\\\\\\\\\\\\
mini_kid_sum_stu 

['mini_kid_sum_stu_timestamp',